# Parallelizing image convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-09-20 03:18:17,468	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 148.3 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-09-20 03:18:56,578	INFO worker.py:1786 -- Started a local Ray instance.


Python version:,3.11.10
Ray version:,2.36.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 86.0, speedup = 1.72


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 172.0, speedup = 0.86


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:39313' processes=4 threads=4, memory=15.62 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-09-20 03:23:19,053 - distributed.worker - ERROR - Compute Failed
Key:       fconv-374674010c29c6d95bdc58692f74321b
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.34479639,  0.80303664, -0.13308151,  0.27603198],
       [ 0.12806245, -0.39985593, -0.54576155,  1.70046146],
       [ 1.13230909,  0.72952181,  0.90557765, -1.28181062],
       [ 0.89393461, -1.53493962,  0.35090589, -1.50846311]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:19,154 - distributed.worker - ERROR - Compute Failed
Key:       fconv-dd7447e5ce2b60de19dd98d84dfa89c7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 03:23:19,257 - distributed.worker - ERROR - Compute Failed
Key:       fconv-37f3c16f91d0fdc27de6fcde828b2f06
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.41937201, -1.08121675,  0.46518257,  0.9107112 ],
       [-1.11455667, -1.10130031,  1.60814691, -1.3327452 ],
       [-0.2809577 ,  0.46245268, -1.26356635, -0.53726099],
       [ 0.22119439, -1.14569298,  1.55339907, -0.1833879 ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:19,351 - distributed.worker - ERROR - Compute Failed
Key:       fconv-6ee32d84a29574d6ff8b05cc1faba473
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 03:23:19,545 - distributed.worker - ERROR - Compute Failed
Key:       fconv-75dd9c0569408862e1e738c2ea60d182
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.34321574, -0.19030729, -1.43676158,  0.72122946],
       [-0.33127491,  0.98704443, -0.64994018,  0.10422607],
       [ 1.20803317,  0.05461822, -0.55609795, -0.57420413],
       [ 1.15606711, -0.31341959,  0.0825348 , -0.09606663]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:19,639 - distributed.worker - ERROR - Compute Failed
Key:       fconv-92edc7540233a2c306cf5da16e911bed
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 03:23:19,830 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ca2643b160c981d9a2f25dc3979a99d9
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.34479639,  0.80303664, -0.13308151,  0.27603198],
       [ 0.12806245, -0.39985593, -0.54576155,  1.70046146],
       [ 1.13230909,  0.72952181,  0.90557765, -1.28181062],
       [ 0.89393461, -1.53493962,  0.35090589, -1.50846311]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:19,925 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2ccd44cd686fc27edea45cccd39d5e34
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 03:23:20,108 - distributed.worker - ERROR - Compute Failed
Key:       fconv-5bad7cccde9b4cf86a031a3892925295
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.53009697,  0.43619401, -0.47394222,  0.86874321],
       [-0.0634436 , -1.24071567, -1.37903294,  0.38202658],
       [-0.64834829,  0.01922922,  1.50603181, -2.41742404],
       [-1.09589037, -1.38533471,  1.51665468,  0.55080037]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:20,199 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0b44a8104bee8635cb00b10e070b4813
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-09-20 03:23:56,043 - distributed.worker - ERROR - Compute Failed
Key:       fconv-32f5db414d7ba3f59b5373bc73db27df
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.41937201, -1.08121675,  0.46518257,  0.9107112 ],
       [-1.11455667, -1.10130031,  1.60814691, -1.3327452 ],
       [-0.2809577 ,  0.46245268, -1.26356635, -0.53726099],
       [ 0.22119439, -1.14569298,  1.55339907, -0.1833879 ]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:56,128 - distributed.worker - ERROR - Compute Failed
Key:       fconv-3de7f95620909d0d2917a21b712bf3c9
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 03:23:56,315 - distributed.worker - ERROR - Compute Failed
Key:       fconv-dfad57262ae48ad00f1d986c611fdd07
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.34321574, -0.19030729, -1.43676158,  0.72122946],
       [-0.33127491,  0.98704443, -0.64994018,  0.10422607],
       [ 1.20803317,  0.05461822, -0.55609795, -0.57420413],
       [ 1.15606711, -0.31341959,  0.0825348 , -0.09606663]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:56,417 - distributed.worker - ERROR - Compute Failed
Key:       fconv-e42035f81afdc858caf0fdedd7c9dc89
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-20 03:23:56,597 - distributed.worker - ERROR - Compute Failed
Key:       fconv-d34d2cd8396bb60bca85eec53366656d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.34479639,  0.80303664, -0.13308151,  0.27603198],
       [ 0.12806245, -0.39985593, -0.54576155,  1.70046146],
       [ 1.13230909,  0.72952181,  0.90557765, -1.28181062],
       [ 0.89393461, -1.53493962,  0.35090589, -1.50846311]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:56,688 - distributed.worker - ERROR - Compute Failed
Key:       fconv-78d94c17cf35dea8109ebc055e091f90
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 38.5, speedup = 3.85


2024-09-20 03:23:56,884 - distributed.worker - ERROR - Compute Failed
Key:       fconv-a8540613912e100ece8c4915d9134193
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.53009697,  0.43619401, -0.47394222,  0.86874321],
       [-0.0634436 , -1.24071567, -1.37903294,  0.38202658],
       [-0.64834829,  0.01922922,  1.50603181, -2.41742404],
       [-1.09589037, -1.38533471,  1.51665468,  0.55080037]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-20 03:23:56,978 - distributed.worker - ERROR - Compute Failed
Key:       fconv-9bd0595e3770aa5cc7869f62908c5c6a
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

In [16]:
client.close()

2024-09-20 03:23:57,021 - distributed.worker - ERROR - Compute Failed
Key:       fconv-e9629d1b314a25eee7d6765f4158081e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.34321574, -0.19030729, -1.43676158,  0.72122946],
       [-0.33127491,  0.98704443, -0.64994018,  0.10422607],
       [ 1.20803317,  0.05461822, -0.55609795, -0.57420413],
       [ 1.15606711, -0.31341959,  0.0825348 , -0.09606663]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''



## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io